In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import matplotlib
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 16})

from astropy.io import fits
from astropy.cosmology import Planck18 as cosmo
import astropy.units as u

In [ ]:
fwhm_star = 0.04 * 2.355 # arcsec

In [ ]:
def fwhm_int_simple(fwhm_obs, fwhm_star):
    return (fwhm_obs**2 + fwhm_star**2) ** 0.5

In [ ]:
CAT_PATH = '/home/alberto/cosmos/ista/COLA1/catalogs/F200W_cat/F200W_det_O3_doublets.fits'
cat = fits.open(CAT_PATH)[1].data
cat.columns

In [ ]:
FWHM_int = fwhm_int_simple(cat['FWHM_WORLD'] * 3600, fwhm_star) * u.arcsec

R_eff = FWHM_int / 2 * cosmo.kpc_proper_per_arcmin(cat['z_O3doublet_combined_n_1'])
R_eff = R_eff.to(u.kpc).value

In [ ]:
z_Arr = cat['z_O3doublet_combined_n_1']

fig, ax = plt.subplots()

mask_COLA1 = cat['NUMBER_1_1'] == 9269

ax.errorbar(z_Arr, R_eff,
            fmt='o')
ax.errorbar(z_Arr[mask_COLA1], R_eff[mask_COLA1],
           marker='*', ms=20, c='r')
ax.errorbar([6.6], fwhm_star / 2 * cosmo.kpc_proper_per_arcmin(6.6).to(u.kpc / u.arcsec).value,
            fmt='o')

ax.set_ylabel(r'$R_{\rm eff}$ [pkpc]')
ax.set_xlabel('Redshift')

plt.show()

In [ ]:
def app_m_to_flux_nu(m):
    """
    :param m: apparent magnitude
    :return: the (observed) flux density
    """
    return 10 ** (-(m + 48.60) / 2.5)


def flux_to_lum(flux, z):
    """
    :param flux: flux
    :param z: redshift
    :return: luminosity
    """
    return flux * 4 * np.pi * cosmo.luminosity_distance(z).to("cm").value ** 2

def flux_nu_to_lum_nu(flux_nu, z):
    """
    :param flux_nu: the (observed) flux density
    :param z: redshift
    :return: the (intrinsic) luminosity density
    """
    return flux_to_lum(flux_nu, z) / (1 + z)

def continuum_powerlaw(wavelength, norm, beta, redshift):
    return norm*(wavelength/(1500.*(1+redshift)))**beta

In [ ]:
from jpasLAEs.utils import flux_to_mag
import sys
sys.path.insert(0, '..')
from dust_correction import SFR_XION_dust_corrected

# Compute SFR
# For now we assume the same value of xi_ion as in Matthee2023.
SFR_C_UV_COLA1 = 43.51
# SFR_C_UV_COLA1 = 43.15
SFR_C_Hb_COLA1 = 41.78

# SFR_C_UV = 43.15
SFR_C_UV = 43.46
SFR_C_Hb = 41.59

SFR = np.zeros(len(cat))
SFR_err_up = np.zeros(len(cat))
SFR_err_down = np.zeros(len(cat))

XION = np.zeros(len(cat))
XION_err_up = np.zeros(len(cat))
XION_err_down = np.zeros(len(cat))

SFR_UV = np.zeros(len(cat))
SFR_UV_err_up = np.zeros(len(cat))
SFR_UV_err_down = np.zeros(len(cat))

for src in range(len(cat)):
    if cat['NUMBER_1_1'][src] == 9269:
        this_SFR_C_Hb = SFR_C_Hb_COLA1
        this_SFR_C_UV = SFR_C_UV_COLA1
        this_Hb = 4.10216
        this_Hb_err = 0.16479
        this_Hg = 2.11797
        # this_Hg =  this_Hb * 0.4
        this_Hg_err = 0.20597
    else:
        this_balmer_dec = 0.47 / 0.4
        this_SFR_C_Hb = SFR_C_Hb
        this_SFR_C_UV = SFR_C_UV
        this_Hb = cat['f_Hb_1'][src]
        this_Hb_err = cat['f_Hb_err_1'][src]
        this_Hg = 0.4 * this_Hb
        this_Hg_err = 0


    this_redshift = cat['z_O3doublet_combined_n_1'][src]
    N_iter = 5000
    this_rand_beta = np.random.normal(cat['beta'][src], cat['beta_err'][src], size=N_iter)
    this_rand_norm = np.random.normal(cat['norm'][src], cat['norm_err'][src], size=N_iter)

    this_obs_1500 = 1500 * (1 + this_redshift)

    args = (this_obs_1500, this_rand_norm, this_rand_beta, this_redshift)
    this_f_UV = continuum_powerlaw(*args) * 1e-20
    
    this_distmod = cosmo.distmod(this_redshift).value

    this_m_UV = flux_to_mag(this_f_UV, this_obs_1500)
    this_m_UV[~np.isfinite(this_m_UV)] = -99

    this_M_UV_Arr = this_m_UV - this_distmod + 2.5 * np.log10(1 + this_redshift)

    this_M_UV = np.nanmedian(this_M_UV_Arr)
    this_M_UV_err = (np.nanpercentile(this_M_UV_Arr, 84) - np.nanpercentile(this_M_UV_Arr, 16)) * 0.5

    SFR_Hb_percs, SFR_UV_percs, XION_percs = SFR_XION_dust_corrected(this_Hg, this_Hg_err,
                                                                     this_Hb, this_Hb_err,
                                                                     this_M_UV, this_M_UV_err,
                                                                     this_redshift, this_SFR_C_Hb,
                                                                     this_SFR_C_UV,
                                                                     N_iter=100000)

    SFR[src] = SFR_Hb_percs[1]
    SFR_err_up[src] = SFR_Hb_percs[2] - SFR_Hb_percs[1]
    SFR_err_down[src] = SFR_Hb_percs[1] - SFR_Hb_percs[0]

    SFR_UV[src] = SFR_UV_percs[1]
    SFR_UV_err_up[src] = SFR_UV_percs[2] - SFR_UV_percs[1]
    SFR_UV_err_down[src] = SFR_UV_percs[1] - SFR_UV_percs[0]

    XION[src] = XION_percs[1]
    XION_err_up[src] = XION_percs[2] - XION_percs[1]
    XION_err_down[src] = XION_percs[1] - XION_percs[0]


print(SFR[mask_COLA1][0], SFR_err_up[mask_COLA1][0], SFR_err_down[mask_COLA1][0])
print(SFR_UV[mask_COLA1][0], SFR_UV_err_up[mask_COLA1][0], SFR_UV_err_down[mask_COLA1][0])
print(XION[mask_COLA1][0], XION_err_up[mask_COLA1][0], XION_err_down[mask_COLA1][0])

In [ ]:
Sigma_SFR = np.log10(SFR / (2 * np.pi * R_eff**2))
Sigma_SFR_err_up = np.log10((SFR + SFR_err_up) / SFR)
Sigma_SFR_err_down = np.log10(SFR / (SFR - SFR_err_down))

Sigma_SFR_UV = np.log10(SFR_UV / (2 * np.pi * R_eff**2))
Sigma_SFR_UV_err_up = np.log10((SFR_UV + SFR_UV_err_up) / SFR_UV)
Sigma_SFR_UV_err_down = np.log10(SFR_UV / (SFR_UV - SFR_UV_err_down))

print('Sigma(Hbeta)', Sigma_SFR[mask_COLA1], Sigma_SFR_err_up[mask_COLA1], Sigma_SFR_err_down[mask_COLA1])
print('Sigma(UV)', Sigma_SFR_UV[mask_COLA1], Sigma_SFR_UV_err_up[mask_COLA1], Sigma_SFR_UV_err_down[mask_COLA1])

In [ ]:
mask_Hb_sn = cat['f_Hb_1'] / cat['f_Hb_err_1'] > 3

fig, ax = plt.subplots()

ax.errorbar(z_Arr[mask_Hb_sn], Sigma_SFR[mask_Hb_sn],
            yerr=[Sigma_SFR_err_up[mask_Hb_sn], Sigma_SFR_err_down[mask_Hb_sn]],
            fmt='o', ls='', label=r'SFR(H$\beta$)', c='r', zorder=91)
ax.errorbar(z_Arr[mask_COLA1], Sigma_SFR[mask_COLA1],
            yerr=[Sigma_SFR_err_up[mask_COLA1], Sigma_SFR_err_down[mask_COLA1]],
            fmt='*', ls='', ms=20, c='r', zorder=901)

ax.errorbar(z_Arr, Sigma_SFR_UV,
            yerr=[Sigma_SFR_UV_err_up, Sigma_SFR_UV_err_down],
            fmt='o', ls='', label=r'SFR(UV)', c='teal', zorder=90)

ax.errorbar(z_Arr[mask_COLA1], Sigma_SFR_UV[mask_COLA1],
            yerr=[Sigma_SFR_UV_err_up[mask_COLA1], Sigma_SFR_UV_err_down[mask_COLA1]],
            fmt='*', ls='', ms=20, c='teal', zorder=900)

#
z_xx = np.linspace(1, 8, 1000)

Sigma_mean = np.nanmean(Sigma_SFR[mask_Hb_sn])
Sigma_std = np.nanstd(Sigma_SFR[mask_Hb_sn])
ax.axhline(Sigma_mean, ls='--', c='r', zorder=10, lw=2)
ax.fill_between(z_xx, np.ones_like(z_xx) * Sigma_mean + Sigma_std,
                np.ones_like(z_xx) * Sigma_mean - Sigma_std,
                color='r', alpha=0.2, lw=0)

mask = (Sigma_SFR > -2) & (Sigma_SFR < 3)
Sigma_mean = np.nanmean(Sigma_SFR[mask])
Sigma_std = np.nanstd(Sigma_SFR[mask])
ax.axhline(Sigma_mean, ls='--', c='teal', zorder=10, lw=2)
ax.fill_between(z_xx, np.ones_like(z_xx) * Sigma_mean + Sigma_std,
                np.ones_like(z_xx) * Sigma_mean - Sigma_std,
                color='teal', alpha=0.2, lw=0)


#

# Shibuya+15
r_0 = 6.9 * (1 + z_xx) ** -1.2
ax.plot(z_xx, np.log10(10 / 2 / (np.pi * r_0**2)),
        lw=3, ls='-', label='Shibuya+15', alpha=0.5)

ax.set_xlim(4.5, 7.5)
ax.set_ylim(-2, 3)

ax.legend(fontsize=11)

ax.set_ylabel(r'$\log_{10}\Sigma_{\rm SFR}$ [$M_\odot$\,yr$^{-1}$\,kpc$^{-2}$]')
ax.set_xlabel('Redshift')

plt.show()

In [ ]:
src = np.where(cat['NUMBER_1_1'] == 9269)[0]

this_SFR_C_UV = SFR_C_UV_COLA1
this_Hb = 4.10216
this_Hb_err = 0.16479
this_Hg = 2.11797
this_Hg_err = 0.20597

this_redshift = 6.59165
this_R_eff = R_eff[mask_COLA1][0]

this_redshift = cat['z_O3doublet_combined_n_1'][src]
N_iter = 5000
this_rand_beta = np.random.normal(cat['beta'][src], cat['beta_err'][src], size=N_iter)
this_rand_norm = np.random.normal(cat['norm'][src], cat['norm_err'][src], size=N_iter)

this_obs_1500 = 1500 * (1 + this_redshift)

args = (this_obs_1500, this_rand_norm, this_rand_beta, this_redshift)
this_f_UV = continuum_powerlaw(*args) * 1e-20

this_distmod = cosmo.distmod(this_redshift).value

this_m_UV = flux_to_mag(this_f_UV, this_obs_1500)
this_m_UV[~np.isfinite(this_m_UV)] = -99

this_M_UV_Arr = this_m_UV - this_distmod + 2.5 * np.log10(1 + this_redshift)

this_M_UV = np.nanmedian(this_M_UV_Arr)
this_M_UV_err = (np.nanpercentile(this_M_UV_Arr, 84) - np.nanpercentile(this_M_UV_Arr, 16)) * 0.5


SFR_Hb_percs, SFR_UV_percs, _ = SFR_XION_dust_corrected(this_Hg, this_Hg_err,
                                                        this_Hb, this_Hb_err,
                                                        this_M_UV, this_M_UV_err,
                                                        this_redshift, this_SFR_C_Hb,
                                                        this_SFR_C_UV,
                                                        N_iter=100000)

N_iter = 10000
this_a = np.random.normal(1.6, 0.3, size=N_iter)
this_b = np.random.normal(0.4, 0.1, size=N_iter)

this_SFR_UV = np.random.normal(SFR_UV_percs[1], (SFR_UV_percs[2] - SFR_UV_percs[0]) * 0.5,
                                 size=N_iter)

this_Sigma_UV = np.log10(this_SFR_UV / (2 * np.pi * this_R_eff**2))

f_esc_Sigma = this_a * (10**this_Sigma_UV / 220)**this_b
print(np.median(f_esc_Sigma), -np.median(f_esc_Sigma) + np.percentile(f_esc_Sigma, [16, 84]))